# K Nearest Neighbors in Python from Scratch

## Import the required libraries
numpy -> for mathematical and matrix computation.<br>
pandas -> handling and manipulating data(s).<br>
matplotlib -> visualising data(s).<br>
collection -> with collection we can do variuos operations but here I use it for Counter (count the number of occurance of same object).<br>
warnings -> warning the user with some message.<br>
random -> used to do some randomising operations.<br>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import warnings
import random

## Load the dataset
here Breast Tumor dataset is used to train the model with existing patient records to check whether the new patient have [begnin or malignant] breast tumor with the help of various features like clump_thickness, unif_cell_size, unif_cell_shape, marg_adhesion, single_epith_cell_size, bare_nuclei, bland_chrom, norm_nucleoli, mitoses, class.<br>
<br>
class<br>
2 -> benign<br>
4 -> malignant<br>

In [2]:
data = pd.read_csv('breast-cancer-wisconsin-data.txt')
data.head()

,id,clump_thickness,unif_cell_size,unif_cell_shape,marg_adhesion,single_epith_cell_size,bare_nuclei,bland_chrom,norm_nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


#### Handle the missing data
Replace the missing data ('?') with -99999 (negative 99999).......Why -99999? This distance is pretty large compare to other data(s). so, -99999 will be considered as OUTLIERS 

In [3]:
data.replace('?', -99999, inplace = True)

#### Remove/Drop 'id' column from the data
As 'id' is unwanted.

In [4]:
data.drop(['id'], axis = 1, inplace = True)

#### Change the type of the data to float and convert the whole data into array || Shuffle the data for randomness

In [5]:
data = data.astype(float).values
random.shuffle(data)

## Split the data for training and testing dictionary
test_size -> size of the testing data.<br>
train_data, test_data -> dictionary used for training and testing the model.<br>
training and testing dictionary consist of two keys [2, 4] 2 - benign tumor; 4 - malignant.

#### Setting parameters and initialising empty dictionary

In [6]:
test_size = 1/4
train_data = {2 : [], 4 : []}
test_data = {2 : [], 4 : []}
train = data[ : -int(test_size * len(data))]  # shape - (466, 10) fetching the data(s) from the main dataset (0 to test_size).
                                              # suppose, the size of the dataset is 100 and the test_size is 1/4 (0.25)
                                              # test_size * len(data) = 0.25 * 100 = 25 
                                              # So, here data[ : -int(test_size * len(data))] represents the first 75 data(s)
test = data[-int(test_size * len(data)) : ]  # shape - (233, 10) featching the remaining data(s) from test_size to last record
                                             # suppose, the size of the dataset is 100 and the test_size is 1/4 (0.25)
                                             # test_size * len(data) = 0.25 * 100 = 25 
                                             # So, here data[-int(test_size * len(data)) : ] represents the last 25 data(s)

#### Spliting the data

In [7]:
for i in train:
    train_data[i[-1]].append(i[:-1])# train_data consist of two keys 2 and 4
                                    # i[-1] is the last column which is 'class' column of the dataset which can only be 2 or 4
                                    # if i[-1] is 2 then except last column all other columns is appended to train_data of key 2
                                    # same for i[-1] is 4
    
for i in test:
    test_data[i[-1]].append(i[:-1])# test_data consist of two keys 2 and 4
                                   # i[-1] is the last column which is 'class' column of the dataset which can only be 2 or 4
                                   # if i[-1] is 2 then except last column all other columns is appended to test_data of key 2
                                   # same for i[-1] is 4

## K Nearest Neighbors
K-NN algorithm stores all the available data and classifies a new data point based on the similarity. This means when new data appears then it can be easily classified into a well suite category by using K- NN algorithm.<br>
K-NN algorithm can be used for Regression as well as for Classification but mostly it is used for the Classification problems.<br>
![alt text](https://static.javatpoint.com/tutorial/machine-learning/images/k-nearest-neighbor-algorithm-for-machine-learning.png "2_1")

![alt text](https://static.javatpoint.com/tutorial/machine-learning/images/k-nearest-neighbor-algorithm-for-machine-learning2.png "2_2")

## Euclidean distance:
![alt text](https://static.javatpoint.com/tutorial/machine-learning/images/k-nearest-neighbor-algorithm-for-machine-learning4.png "2_3")

## Working of the algorithm:<br>
In K-NN, when a new datapoint is introduced in the model,<br>
......The algorithm will calculate the Euclidean distances from this point to each point. And store all the distances as a list.<br>
......Then it sorts the list and fetch first k elements from the sorted list. (k - Number of neighbors).<br>
......It counts the number of occurences of each classes and store it as list of tuples. list [ tuple ( class, number of occurence ), ... ]<br> 
where the list is sorted(descending order of number of occurence of the class)<br>
......The first element of the list is the most occured class.<br>
......So, the new datapoint belongs to this class.<br>

Assume, here group and class are same

In [8]:
def k_nearest_neighbors(data, predict, k = 5): # k - number of neighbors.
    if len(data) >= k:
        warnings.warn('The data consist of more classes than K...Please set the K value greater than the number of classes...')
        
    distances = [] # setting empty list to store all the distances
    
    for labels in data:
        for coordinates in data[labels]:
            euclidean_distance = np.linalg.norm(np.array(coordinates) - np.array(predict)) #calculating Euclidean distance
            distances.append((euclidean_distance, labels)) #appending each distances, labels to the list.
            
    groups = [d[1] for d in sorted(distances)[:k]]           # for d in sorted(distances)[:k]:
                                                             #     groups = d[1]
    group = Counter(groups).most_common(1)[0][0]  # fetching the more occurence group/class       
    confidence = Counter(groups).most_common(1)[0][1] / k  # how likely the patient belongs to some group/class (probability) 
    
    return group, confidence

In [9]:
correct = 0  #number of correct predictions
total = 0  #number of total predictions

for labels in test_data:  # Runs number of classes times.
    for coordinates in test_data[labels]:  #Runs number of data inside each class.
        
        group, confidence = k_nearest_neighbors(train_data, coordinates, k = 5) # Passing the training data and new datapoints 
                                                                                # to the K-NN algorithm.
                                                                            # coordinates is the new datapoint to be predicted.
        if labels == group:    # group is the prediction of new datapoint (predicted class) which is compared with real class
                               # of the new datapoint.
                
            correct += 1       # if both are same then correct is added by 1
            
        else:
            print(confidence)  # if both are not same then print the probability of incorrectness in prediction
            
        total += 1
        
print("Accuracy = ", correct / total, "%") # Accuracy of the model

0.8
0.8
1.0
0.6
0.8
Accuracy =  0.9712643678160919 %


# Thank you.......................................!!